NEXT WORD PREDICTION PROJECT BY HARSHIT


In [23]:
import tensorflow as tf
import string
import requests

In [33]:
response=requests.get("http://www.gutenberg.org/cache/epub/5200/pg5200.txt")

In [34]:
response.text[:1500]

'\ufeffThe Project Gutenberg EBook of Metamorphosis, by Franz Kafka\r\nTranslated by David Wyllie.\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **\r\n**     Please follow the copyright guidelines in this file.     **\r\n\r\n\r\nTitle: Metamorphosis\r\n\r\nAuthor: Franz Kafka\r\n\r\nTranslator: David Wyllie\r\n\r\nRelease Date: August 16, 2005 [EBook #5200]\r\nFirst posted: May 13, 2002\r\nLast updated: May 20, 2012\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK METAMORPHOSIS ***\r\n\r\n\r\n\r\n\r\nCopyright (C) 2002 David Wyllie.\r\n\r\n\r\n\r\n\r\n\r\n  Metamorphosis\r\n  Franz Kafka\r\n\r\nTranslated by David Wyllie\r\n\r\n\r\n\r\nI\r\n\r\n\r\nOne morning, when Gregor

Split the data set into lines


In [35]:
data = response.text.split('\n')
data[0]

'\ufeffThe Project Gutenberg EBook of Metamorphosis, by Franz Kafka\r'

In [36]:
data = data[253:]
data[0]

'away from the bed, bend down with the load and then be patient and\r'

In [37]:
len(data)

2110

In [38]:
data = " ".join(data)
data[:1000]


'away from the bed, bend down with the load and then be patient and\r careful as he swang over onto the floor, where, hopefully, the\r little legs would find a use.  Should he really call for help\r though, even apart from the fact that all the doors were locked?\r Despite all the difficulty he was in, he could not suppress a smile\r at this thought.\r \r After a while he had already moved so far across that it would have\r been hard for him to keep his balance if he rocked too hard.  The\r time was now ten past seven and he would have to make a final\r decision very soon.  Then there was a ring at the door of the flat.\r "That\'ll be someone from work", he said to himself, and froze very\r still, although his little legs only became all the more lively as\r they danced around.  For a moment everything remained quiet.\r "They\'re not opening the door", Gregor said to himself, caught in\r some nonsensical hope.  But then of course, the maid\'s firm steps\r went to the door as ever and o

So here we can see that after passing data to clean_text we get the data in the required format without punctuations and special characters.


In [39]:
def clean_text(doc):
 tokens = doc.split()
 table = str.maketrans('', '', string.punctuation)
 tokens = [w.translate(table) for w in tokens]
 tokens = [word for word in tokens if word.isalpha()]
 tokens = [word.lower() for word in tokens]
 return tokens
tokens = clean_text(data)
print(tokens[:50])

['away', 'from', 'the', 'bed', 'bend', 'down', 'with', 'the', 'load', 'and', 'then', 'be', 'patient', 'and', 'careful', 'as', 'he', 'swang', 'over', 'onto', 'the', 'floor', 'where', 'hopefully', 'the', 'little', 'legs', 'would', 'find', 'a', 'use', 'should', 'he', 'really', 'call', 'for', 'help', 'though', 'even', 'apart', 'from', 'the', 'fact', 'that', 'all', 'the', 'doors', 'were', 'locked', 'despite']


In [40]:
len(tokens)

22607

In [43]:
length = 50 + 1
lines = []
for i in range(length, len(tokens)):
 seq = tokens[i-length:i]
 line = ' '.join(seq)
 lines.append(line)
 if i > 200000:
   break
print(len(lines))


22556


Build LSTM Model and Prepare X and y


In [44]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [46]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[0]


array([ 103,   29,    1,  245, 2883,   98,   14,    1, 1435,    3,   48,
         30,  618,    3,  756,   13,    6, 1434,  107,  165,    1,  149,
         86, 2880,    1,   78,  225,   21,  530,   12,  156,  193,    6,
        142,  754,   17,  180,  116,   49, 1433,   29,    1,  753,   11,
         26,    1,  455,   58,  617,  329])

In [47]:
vocab_size = len(tokenizer.word_index) + 1

In [48]:
y = to_categorical(y, num_classes=vocab_size)

In [49]:
seq_length = X.shape[1]
seq_length

50

LSTM Model

In [50]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            144250    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 2885)              291385    
Total params: 586,535
Trainable params: 586,535
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [53]:
model.fit(X, y, batch_size = 256, epochs = 100)

Epoch 1/100
89/89 [==============================] - 37s 373ms/step - loss: 6.6391 - accuracy: 0.0496
Epoch 2/100
89/89 [==============================] - 33s 371ms/step - loss: 6.1889 - accuracy: 0.0540
Epoch 3/100
89/89 [==============================] - 33s 370ms/step - loss: 6.1645 - accuracy: 0.0540
Epoch 4/100
89/89 [==============================] - 33s 368ms/step - loss: 6.0556 - accuracy: 0.0540
Epoch 5/100
89/89 [==============================] - 33s 371ms/step - loss: 5.9206 - accuracy: 0.0557
Epoch 6/100
89/89 [==============================] - 33s 371ms/step - loss: 5.8009 - accuracy: 0.0575
Epoch 7/100
89/89 [==============================] - 33s 372ms/step - loss: 5.7255 - accuracy: 0.0606
Epoch 8/100
89/89 [==============================] - 33s 370ms/step - loss: 5.6665 - accuracy: 0.0660
Epoch 9/100
89/89 [==============================] - 33s 372ms/step - loss: 5.6145 - accuracy: 0.0709
Epoch 10/100
89/89 [==============================] - 33s 371ms/step - loss: 5.567

In [63]:
seed_text=lines[12343]
seed_text

'condition seemed serious enough to remind even his father that gregor despite his current sad and revolting form was a family member who could not be treated as an enemy on the contrary as a family there was a duty to swallow any revulsion for him and to be patient just'

In [64]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []
  
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')
    y_predict = model.predict_classes(encoded)
    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
        return ' '.join(text)



In [65]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


We have got a accuracy of 46%. To increase the accuracy we can increase the number of epochs or we can consider the entire data for training. For this model we have only considered 1/4th of the
data for training